# AutoShorts 간단 테스트

각 모듈을 독립적으로 테스트합니다.

## 필수 조건
- `.env` 파일에 API 키 설정
- 가상환경 활성화: `source kvenv/bin/activate`
- 패키지 설치: `pip install -r backend/requirements.txt`

In [1]:
!pip install -r backend/requirements.txt


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# 1. 환경 설정
import sys
import json
from pathlib import Path
from datetime import datetime

# Python path 설정
project_root = Path.cwd()
backend_path = project_root / "backend"

if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))

print(f"✅ 프로젝트 루트: {project_root}")
print(f"✅ Backend path: {backend_path}")
print(f"✅ Python path 설정 완료")

✅ 프로젝트 루트: /Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI
✅ Backend path: /Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI/backend
✅ Python path 설정 완료


In [3]:
# 2. Config 로드 및 API 키 확인
# 중요: .env 파일을 변경한 경우 모듈을 reload해야 합니다
import importlib
import app.config
importlib.reload(app.config)
from app.config import settings

print("=" * 60)
print("API 키 상태")
print("=" * 60)

api_checks = {
    "OpenAI (필수)": settings.OPENAI_API_KEY,
    "ElevenLabs (음성+음악)": settings.ELEVENLABS_API_KEY,
    "ComfyUI URL": settings.COMFY_URL,
    "Redis URL": settings.REDIS_URL,
}

for name, value in api_checks.items():
    status = "✅" if value else "❌"
    display_value = f"{value[:15]}...{value[-10:]}" if value and len(value) > 30 else value
    print(f"{name:30s}: {status} {display_value if value else ''}")

print("\n" + "=" * 60)
print(f"Workflow 경로: {settings.COMFY_WORKFLOW}")
print("=" * 60)

API 키 상태
OpenAI (필수)                   : ✅ sk-proj-TwhsAmu...SVdVmi6z4A
ElevenLabs (음성+음악)            : ✅ sk_ee50fbab9484...b400b42392
ComfyUI URL                   : ✅ http://localhost:8188
Redis URL                     : ✅ redis://localhost:6379/0

Workflow 경로: backend/app/providers/images/workflows/flux_omni_lora.json


## 테스트 1: CSV 생성 (GPT-4o-mini)

In [4]:
# 3. CSV 생성 테스트
from app.utils.csv_to_json import generate_csv_from_prompt

# 테스트 파라미터
test_prompt = "우주를 탐험하는 용감한 고양이"
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
run_id = f"{timestamp}_테스트"

print("=" * 60)
print("CSV 생성 테스트")
print("=" * 60)
print(f"Run ID: {run_id}")
print(f"프롬프트: {test_prompt}")
print(f"캐릭터 수: 2명")
print(f"장면 수: 3개")
print("\n🔄 CSV 생성 중...\n")

try:
    csv_path = generate_csv_from_prompt(
        run_id=run_id, prompt=test_prompt, num_characters=2, num_cuts=3, mode="story"
    )

    print(f"\n✅ CSV 생성 성공: {csv_path}")

    # CSV 내용 확인
    import pandas as pd

    df = pd.read_csv(csv_path)
    print(f"\n생성된 데이터 ({len(df)}행):")
    print(df.to_string(index=False))

except Exception as e:
    print(f"\n❌ CSV 생성 실패: {e}")
    import traceback

    traceback.print_exc()

CSV 생성 테스트
Run ID: 20251105_1437_테스트
프롬프트: 우주를 탐험하는 용감한 고양이
캐릭터 수: 2명
장면 수: 3개

🔄 CSV 생성 중...


✅ CSV 생성 성공: app/data/outputs/20251105_1437_테스트/plot.csv

생성된 데이터 (6행):
scene_id char_id char_name                     text text_type   emotion subtitle_position  duration_ms
 scene_1  char_1     우주고양이       나는 오늘 우주를 탐험하러 간다!  dialogue     happy               top         5000
 scene_1  char_2      로봇친구 준비 완료! 우주선은 출발할 준비가 되었어!  dialogue   excited            bottom         5000
 scene_2  char_1     우주고양이          와, 별들이 정말 아름다워!  dialogue surprised               top         5000
 scene_2  char_2      로봇친구 네, 별들은 우주에서 가장 빛나는 보석이지!  dialogue     happy            bottom         5000
 scene_3  char_1     우주고양이     다음 목적지는 그 신비로운 행성이야!  dialogue   excited               top         5000
 scene_3  char_2      로봇친구           모험이 시작되네! 준비해!  dialogue     happy            bottom         5000


## 테스트 2: CSV → JSON 변환

In [25]:
# 4. CSV → JSON 변환 테스트
from app.utils.csv_to_json import csv_to_json

print("=" * 60)
print("CSV → JSON 변환 테스트")
print("=" * 60)
print("\n🔄 JSON 변환 중...\n")

try:
    json_path = csv_to_json(
        csv_path=str(csv_path),
        run_id=run_id,
        art_style="파스텔 수채화",
        music_genre="ambient",
    )

    print(f"✅ JSON 생성 성공: {json_path}")

    # JSON 구조 확인
    with open(json_path, "r", encoding="utf-8") as f:
        layout = json.load(f)

    print(f"\n프로젝트 ID: {layout['project_id']}")
    print(f"캐릭터 수: {len(layout['characters'])}명")
    print(f"씬 수: {len(layout['scenes'])}개")
    print(f"총 길이: {layout['timeline']['total_duration_ms']}ms")

    print("\n캐릭터 정보:")
    for char in layout["characters"]:
        print(f"  - {char['name']} (ID: {char['char_id']}, Seed: {char['seed']})")

except Exception as e:
    print(f"\n❌ JSON 변환 실패: {e}")
    import traceback

    traceback.print_exc()

CSV → JSON 변환 테스트

🔄 JSON 변환 중...

✅ JSON 생성 성공: app/data/outputs/20251105_1432_테스트/layout.json

프로젝트 ID: 20251105_1432_테스트
캐릭터 수: 2명
씬 수: 3개
총 길이: 15000ms

캐릭터 정보:
  - 코스모 (ID: char_1, Seed: 1002)
  - 스타 (ID: char_2, Seed: 1003)


## 테스트 3: 음성 생성 (ElevenLabs TTS)

ElevenLabs TTS API를 사용하여 음성을 생성합니다.

In [7]:
# 5. 음성 생성 테스트 (ElevenLabs TTS)
# 중요: settings와 client 모듈을 다시 로드하여 최신 코드 사용
import importlib
import app.config
import app.providers.tts.elevenlabs_client

importlib.reload(app.config)
importlib.reload(app.providers.tts.elevenlabs_client)

from app.config import settings
from app.providers.tts.elevenlabs_client import ElevenLabsClient

print("=" * 60)
print("음성 생성 테스트 (ElevenLabs TTS)")
print("=" * 60)

# API 키 확인
print(f"\n🔍 현재 API 키: {settings.ELEVENLABS_API_KEY[:15]}...{settings.ELEVENLABS_API_KEY[-10:]}")
print(f"   키 길이: {len(settings.ELEVENLABS_API_KEY)} 문자")

if settings.ELEVENLABS_API_KEY:
    try:
        # API 키 전달하여 클라이언트 초기화
        client = ElevenLabsClient(api_key=settings.ELEVENLABS_API_KEY)
        print("\n✅ ElevenLabs TTS 클라이언트 초기화 성공")

        # 테스트 케이스
        test_cases = [
            {
                "text": "안녕하세요! 저는 우주를 탐험하는 고양이입니다.",
                "filename": "test_korean.mp3"
            },
            {
                "text": "Hello! This is an English TTS test.",
                "filename": "test_english.mp3"
            }
        ]

        print(f"\n총 {len(test_cases)}개 테스트 케이스\n")

        for idx, test in enumerate(test_cases, 1):
            print(f"[테스트 {idx}/{len(test_cases)}]")
            print(f"텍스트: {test['text']}")
            print("🔄 음성 생성 중...\n")

            voice_path = client.generate_speech(
                text=test['text'],
                voice_id="default",
                emotion="neutral",
                output_filename=f"backend/app/data/outputs/{run_id}/{test['filename']}"
            )

            # 파일 크기 확인
            file_size = Path(voice_path).stat().st_size / 1024

            print(f"✅ 생성 성공: {voice_path}")
            print(f"   파일 크기: {file_size:.2f} KB\n")

            # 오디오 재생
            from IPython.display import Audio, display
            if Path(voice_path).exists():
                display(Audio(filename=voice_path))
            
            print("-" * 60 + "\n")

        print("\n🎉 모든 TTS 테스트 완료!")

    except Exception as e:
        print(f"\n❌ TTS 테스트 실패: {e}")
        print(f"\n💡 트러블슈팅:")
        print(f"   1. Kernel → Restart Kernel 실행")
        print(f"   2. Cell 1부터 순서대로 다시 실행")
        print(f"   3. 또는 독립 스크립트 실행: python test_tts_simple.py")
        import traceback
        traceback.print_exc()
else:
    print("\n⚠️  ELEVENLABS_API_KEY가 설정되지 않음")
    print("\n💡 Stub 모드를 사용하려면:")
    print("   - StubTTSClient 사용")
    print("   - API 비용 없이 테스트 가능")

음성 생성 테스트 (ElevenLabs TTS)

🔍 현재 API 키: sk_ee50fbab9484...b400b42392
   키 길이: 51 문자

✅ ElevenLabs TTS 클라이언트 초기화 성공

총 2개 테스트 케이스

[테스트 1/2]
텍스트: 안녕하세요! 저는 우주를 탐험하는 고양이입니다.
🔄 음성 생성 중...

✅ 생성 성공: backend/app/data/outputs/20251105_1437_테스트/test_korean.mp3
   파일 크기: 57.19 KB



------------------------------------------------------------

[테스트 2/2]
텍스트: Hello! This is an English TTS test.
🔄 음성 생성 중...

✅ 생성 성공: backend/app/data/outputs/20251105_1437_테스트/test_english.mp3
   파일 크기: 37.59 KB



------------------------------------------------------------


🎉 모든 TTS 테스트 완료!


## 테스트 4: 이미지 생성 (ComfyUI)

**주의**: ComfyUI 서버가 실행 중이어야 합니다 (`http://localhost:8188`)

In [9]:
# 7. 이미지 생성 테스트
print("=" * 60)
print("이미지 생성 테스트 (ComfyUI)")
print("=" * 60)

# ComfyUI 연결 확인
import requests

try:
    response = requests.get(f"{settings.COMFY_URL}/system_stats", timeout=2)
    print(f"\n✅ ComfyUI 서버 연결 성공: {settings.COMFY_URL}")
    comfyui_available = True
except:
    print(f"\n⚠️  ComfyUI 서버 연결 실패: {settings.COMFY_URL}")
    print("Stub 모드로 더미 이미지 생성")
    comfyui_available = False

if comfyui_available:
    from app.providers.images.comfyui_client import ComfyUIClient

    try:
        client = ComfyUIClient(base_url=settings.COMFY_URL)

        # 첫 번째 캐릭터로 이미지 생성
        char = layout["characters"][0]
        prompt = f"파스텔 수채화, {char['persona']}, cute cat, standing"

        print(f"\n캐릭터: {char['name']}")
        print(f"프롬프트: {prompt}")
        print(f"Seed: {char['seed']}")

        print("\n🔄 이미지 생성 중 (시간이 걸릴 수 있습니다)...\n")

        image_path = client.generate_image(
            prompt=prompt,
            seed=char["seed"],
            output_prefix=f"backend/app/data/outputs/{run_id}/test_image",
        )

        print(f"✅ 이미지 생성 성공: {image_path}")

        # 이미지 표시
        from IPython.display import Image, display

        if Path(image_path).exists():
            display(Image(filename=image_path, width=400))

    except Exception as e:
        print(f"\n❌ 이미지 생성 실패: {e}")
        import traceback

        traceback.print_exc()
else:
    print("\n💡 ComfyUI를 사용하려면:")
    print("   1. ComfyUI 설치 및 실행")
    print("   2. http://localhost:8188 접속 확인")
    print("   3. 노트북 재실행")

이미지 생성 테스트 (ComfyUI)

⚠️  ComfyUI 서버 연결 실패: http://localhost:8188
Stub 모드로 더미 이미지 생성

💡 ComfyUI를 사용하려면:
   1. ComfyUI 설치 및 실행
   2. http://localhost:8188 접속 확인
   3. 노트북 재실행


## 테스트 완료

### 생성된 파일 확인

In [ ]:
# 8. 생성된 파일 확인
output_dir = Path(f"backend/app/data/outputs/{run_id}")

print("=" * 60)
print(f"생성된 파일 목록")
print("=" * 60)

if output_dir.exists():
    files = sorted(output_dir.glob("*"))

    for file in files:
        size_kb = file.stat().st_size / 1024
        print(f"{file.name:40s} {size_kb:>10.2f} KB")

    print(f"\n총 {len(files)}개 파일")
    print(f"\n디렉토리: {output_dir}")
else:
    print("출력 디렉토리가 없습니다.")

print("\n" + "=" * 60)
print("✅ 모든 테스트 완료")
print("=" * 60)